In [1]:
import numpy as np
import lysis
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FFMpegWriter, FuncAnimation
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.patches import Patch

from IPython.display import HTML
plt.rcParams["animation.html"] = "jshtml"

In [2]:
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code="2023-01-31-1200")
e.read_file()
rng = np.random.default_rng()
f_file_code = ".f-array.dat"
p_file_code = ".p.npy"

In [3]:
print(e)

experiment_code : 2023-01-31-1200
data_filenames  : unbinding_time    : tsectPA.dat
                  lysis_time        : lysismat.dat
                  total_lyses       : lenlysisvect.dat
                  degradation_state : deg.p.npy
                  molecule_location : m_loc.p.npy
                  molecule_state    : m_bound.p.npy
                  save_time         : tsave.p.npy
micro_params    : None
macro_params    : binding_rate       : 0.1
                  pore_size          : 0.00010135
                  diffusion_coeff    : 5e-07
                  binding_sites      : 427
                  forced_unbind      : 0.0852
                  average_bind_time  : 27.8
                  grid_node_distance : 1.0862
                  cols               : 10
                  rows               : 13
                  full_row           : 29
                  xz_row             : 19
                  total_edges        : 367
                  total_fibers       : 280
                

In [4]:
f_deg = np.fromfile(os.path.join(e.os_path, "deg" + f_file_code))
f_mol_location = np.fromfile(os.path.join(e.os_path, "m_loc" + f_file_code), dtype=np.int32)
f_mol_status = np.fromfile(os.path.join(e.os_path, "m_bound" + f_file_code), dtype=np.int32)
f_tsave = np.fromfile(os.path.join(e.os_path, "tsave" + f_file_code))
f_deg = f_deg.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_edges)
f_mol_location = f_mol_location.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_molecules) - 1
f_mol_status = f_mol_status.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_molecules)
f_tsave = f_tsave.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves)

In [5]:
p_deg = np.load(os.path.join(e.os_path, "deg" + p_file_code))
p_mol_location = np.load(os.path.join(e.os_path, "m_loc" + p_file_code))
p_mol_status = np.load(os.path.join(e.os_path, "m_bound" + p_file_code))
p_tsave = np.load(os.path.join(e.os_path, "tsave" + p_file_code))
p_deg = np.expand_dims(p_deg, axis=0)
p_mol_location = np.expand_dims(p_mol_location, axis=0)
p_mol_status = np.expand_dims(p_mol_status, axis=0)
p_tsave = np.expand_dims(p_tsave, axis=0)
true_edge = np.full(e.macro_params.rows*e.macro_params.full_row,
                    True,
                    np.bool_)
for k in range(e.macro_params.cols):
    true_edge[(e.macro_params.rows-1)*e.macro_params.full_row + 3*k] = False

In [6]:
f_mol_status = f_mol_status.astype(np.bool_)
f_mapped_deg = -f_deg
f_mapped_deg[f_deg == 0] = e.macro_params.total_time * 2
f_mapped_deg[f_deg == -1] = 0
p_mapped_deg = p_deg[:, :, true_edge]
for r in range(e.macro_params.total_trials):
    for s in range(e.macro_params.number_of_saves):
        p_mapped_deg[r, s][p_mapped_deg[r, s] > p_tsave[r, s]] = e.macro_params.total_time * 2

In [7]:
def plot_coords(i, j):
    x = j
    y = i
    if j % 3 == 0:
        return x / 3.0, y + 0.5
    if j % 3 == 1:
        return (x - 1) / 3.0, y
    if j % 3 == 2:
        return (x - 2) / 3.0 + 0.5, y

In [8]:
f_edge_index = np.empty(e.macro_params.total_edges, dtype=tuple)
for k in range(e.macro_params.total_edges):
    f_edge_index[k] = lysis.from_fortran_edge_index(k, e.macro_params.rows, e.macro_params.cols)
p_edge_index = np.empty(e.macro_params.rows*e.macro_params.full_row, dtype=tuple)
for k in range(e.macro_params.rows*e.macro_params.full_row):
    p_edge_index[k] = np.unravel_index(k, shape=(e.macro_params.rows, e.macro_params.full_row))

In [9]:
f_x_f = np.empty(e.macro_params.total_edges, dtype=float)
f_y_f = np.empty(e.macro_params.total_edges, dtype=float)
p_x_f = np.empty(e.macro_params.total_edges, dtype=float)
p_y_f = np.empty(e.macro_params.total_edges, dtype=float)
for k in range(e.macro_params.total_edges):
    i, j = f_edge_index[k]
    f_x_f[k], f_y_f[k] = plot_coords(i, j)
    i, j = p_edge_index[true_edge][k]
    p_x_f[k], p_y_f[k] = plot_coords(i, j)

In [10]:
d_x = (rng.random(size=e.macro_params.total_molecules) - 0.5) / 2.5
d_y = (rng.random(size=e.macro_params.total_molecules) - 0.5) / 2.5
f_x_m = np.empty((e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_molecules), dtype=float)
f_y_m = np.empty((e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_molecules), dtype=float)
p_x_m = np.empty((e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_molecules), dtype=float)
p_y_m = np.empty((e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_molecules), dtype=float)
for run in range(e.macro_params.total_trials):
    for t in range(e.macro_params.number_of_saves):
        for k in range(e.macro_params.total_molecules):
            i, j = f_edge_index[f_mol_location[run, t, k]]
            f_x_m[run, t, k], f_y_m[run, t, k] = plot_coords(i, j)
            i, j = p_edge_index[p_mol_location[run, t, k]]
            p_x_m[run, t, k], p_y_m[run, t, k] = plot_coords(i, j)
        f_x_m[run, t] += d_x
        f_y_m[run, t] += d_y
        p_x_m[run, t] += d_x
        p_y_m[run, t] += d_y

In [14]:
ne = f_edge_index[f_mol_location[0]] != p_edge_index[p_mol_location[0]]
np.argwhere(ne)

array([[100,  21],
       [101,  21],
       [102,  21],
       [103,  21],
       [104,  21],
       [105,  21],
       [106,  21],
       [107,  21],
       [108,  21],
       [109,  21],
       [110,  21],
       [111,  21],
       [112,  21],
       [113,  21],
       [114,  21],
       [115,  21],
       [116,  21],
       [117,  21],
       [118,  21],
       [119,  21],
       [120,  21],
       [121,  21],
       [122,  21],
       [123,  21],
       [124,  21],
       [124,  47],
       [125,  21],
       [125,  47],
       [126,  21],
       [126,  47],
       [127,  21],
       [127,  47],
       [128,  21],
       [128,  47],
       [129,  21],
       [129,  47],
       [130,  21],
       [130,  47],
       [131,  21],
       [131,  47],
       [132,  21],
       [132,  47],
       [133,  21],
       [133,  47],
       [134,  21],
       [134,  47],
       [135,  21],
       [135,  47],
       [136,  21],
       [136,  47],
       [137,  21],
       [137,  47],
       [138,

In [26]:
f_edge_index[f_mol_location[0,100,21]], p_edge_index[p_mol_location[0,100,21]]

((5, 25), (7, 6))

In [17]:
f_mol_tracker = np.fromfile(os.path.join(e.os_path, "m_tracker" + f_file_code), dtype=np.int32)

In [21]:
f_mol_tracker = f_mol_tracker.reshape(f_mol_tracker.size // 2, 2)

In [27]:
p_mol_tracker = np.load(os.path.join(e.os_path, "m_tracker" + p_file_code))

In [30]:
p_mol_tracker.shape, f_mol_tracker.shape

((49026, 2), (38246, 2))

In [39]:
np.argwhere(f_edge_index[f_mol_tracker[:38246, 1]] != p_edge_index[p_mol_tracker[:38246,1]])

array([[27265],
       [27266],
       [27267],
       ...,
       [38243],
       [38244],
       [38245]])

In [45]:
f_edge_index[143], 290438*e.macro_params.time_step

((4, 24), 99.44425277516665)

In [43]:
for i in range(27260,38246):
    print(f_mol_tracker[i, 0], 
          f_edge_index[f_mol_tracker[i, 1]],
          p_mol_tracker[i, 0],
          p_edge_index[p_mol_tracker[i,1]])

290395 (5, 26) 290395 (5, 26)
290409 (5, 25) 290409 (5, 25)
290417 (5, 24) 290417 (5, 24)
290423 (5, 25) 290423 (5, 25)
290438 (4, 24) 290438 (4, 24)
290460 (5, 25) 290440 (5, 26)
290464 (4, 24) 290442 (5, 25)
290467 (5, 25) 290443 (5, 24)
290469 (4, 24) 290450 (6, 26)
290478 (5, 25) 290459 (6, 25)
290480 (4, 24) 290460 (6, 26)
290501 (5, 25) 290464 (5, 27)
290509 (4, 24) 290467 (5, 26)
290510 (5, 25) 290469 (5, 25)
290512 (4, 24) 290475 (5, 26)
290515 (5, 25) 290478 (5, 28)
290517 (4, 24) 290480 (4, 27)
290519 (5, 25) 290501 (4, 28)
290527 (4, 24) 290507 (4, 26)
290533 (5, 25) 290509 (3, 24)
290541 (4, 24) 290510 (3, 23)
290547 (5, 25) 290512 (3, 25)
290558 (4, 24) 290515 (2, 24)
290565 (5, 25) 290517 (2, 25)
290571 (4, 24) 290519 (2, 26)
290574 (5, 25) 290524 (2, 28)
290578 (4, 24) 290527 (1, 27)
290587 (5, 25) 290530 (2, 28)
290590 (4, 24) 290533 (2, 26)
290592 (5, 25) 290534 (1, 24)
290597 (4, 24) 290541 (2, 25)
290601 (5, 25) 290547 (2, 24)
290606 (4, 24) 290558 (2, 25)
290607 (5,

In [ ]:
np.append(p_tsave, f_tsave, axis=0).T

In [ ]:
p_mol_location